In [1]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
reviewFoodComment = pd.read_csv('https://raw.githubusercontent.com/TanakornSunny/MADT8101_customer_analytic/main/05_voice_of_customer/wongnai_review.csv')

In [3]:
reviewFoodComment

,Review
0,คาเฟ่ แห่งหนึ่งย่านบางใหญ่ มีที่จอดรถหน้าร้าน ...
1,ร้านนี้เด่นที่เป็นร้านคาเฟ่กาแฟที่มีขนมไทยขายด...
2,ร้านกาแฟร้านนี้เราทานตั้งแต่ตอนยังไม่ต่อเติมหร...
3,วันนี้จะมาแนะนำ ร้านกาแฟเทพๆ ย่านถนนนิมมาน ร้า...
4,ร้านกาแฟกรุงเทพ บรรยากาศดี มีกาแฟ เครื่องดื่มห...
5,แนะนำให้มาลองกินกาแฟร้านนี้ เป็นร้านกาแฟเล็ก ๆ...
6,ร้านอาหารอิตาเลียนที่มีทั้งอาหารเวียดนาม อาหาร...
7,พิซซ่าหน้าชีสล้วนๆ อร่อย หวานมัน นัวเนียในปาก ...
8,บอกเลยว่าพิซซ่าเด็ดจริง! ถ้าใครเคยมีโอกาสไปทาน...
9,ร้านพิซซ่าที่มีเจ้าของเป็นคนเกาหลี ดีกรีแชมป์ห...


In [4]:
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['',' ','\n','ร้าน','(', ')']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
    merged = ''
    words = pythainlp.word_tokenize(str(sentence), engine='newmm')
    for word in words:
        if word not in screening_words:
            merged += ',' + word
    return merged[1:]
        
    

In [5]:
reviewFoodComment['Review_tokenized'] = reviewFoodComment['Review'].apply(lambda x: tokenize_with_space(x))

In [6]:
reviewFoodComment

,Review,Review_tokenized
0,คาเฟ่ แห่งหนึ่งย่านบางใหญ่ มีที่จอดรถหน้าร้าน ...,"คาเฟ่,ย่าน,ที่จอดรถ,หน้า,จอด,คัน,ลานจอดรถ,คาเฟ..."
1,ร้านนี้เด่นที่เป็นร้านคาเฟ่กาแฟที่มีขนมไทยขายด...,"เด่น,คาเฟ่,กาแฟ,ขนมไทย,ขาย,แพ็คเกจ,เครื่องดื่ม..."
2,ร้านกาแฟร้านนี้เราทานตั้งแต่ตอนยังไม่ต่อเติมหร...,"ร้านกาแฟ,ทาน,ตอน,ต่อเติม,ห้อง,แอร์,สวย,แบบนี้,..."
3,วันนี้จะมาแนะนำ ร้านกาแฟเทพๆ ย่านถนนนิมมาน ร้า...,"แนะนำ,ร้านกาแฟ,เทพ,ย่าน,ถนน,นิมมาน,หน้า,นิมมาน..."
4,ร้านกาแฟกรุงเทพ บรรยากาศดี มีกาแฟ เครื่องดื่มห...,"ร้านกาแฟ,กรุงเทพ,บรรยากาศ,ดี,กาแฟ,เครื่องดื่ม,..."
5,แนะนำให้มาลองกินกาแฟร้านนี้ เป็นร้านกาแฟเล็ก ๆ...,"แนะนำ,ลอง,กิน,กาแฟ,ร้านกาแฟ,คน,ซื้อ,เรื่อย ๆ,ห..."
6,ร้านอาหารอิตาเลียนที่มีทั้งอาหารเวียดนาม อาหาร...,"ร้านอาหาร,อิตาเลียน,อาหาร,เวียดนาม,อาหารจีน,อา..."
7,พิซซ่าหน้าชีสล้วนๆ อร่อย หวานมัน นัวเนียในปาก ...,"พิซซ่า,หน้า,ชีส,อร่อย,หวาน,นัวเนีย,ปาก,กิน,ตอน..."
8,บอกเลยว่าพิซซ่าเด็ดจริง! ถ้าใครเคยมีโอกาสไปทาน...,"พิซซ่า,เด็ด,!,มีโอกาส,ทาน,อิตาลี,ทาน,พิซซ่า,แท..."
9,ร้านพิซซ่าที่มีเจ้าของเป็นคนเกาหลี ดีกรีแชมป์ห...,"พิซซ่า,มีเจ้าของ,คน,เกาหลี,ดีกรี,แชมป์,หมุน,แป..."


In [7]:
documents = reviewFoodComment['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [8]:
print(dictionary.token2id.keys())

dict_keys(['คัน', 'คาเฟ่', 'จอด', 'จุด', 'ตกแต่ง', 'ถ่ายรูป', 'ที่จอดรถ', 'ย่าน', 'ลานจอดรถ', 'สวน', 'สวยงาม', 'หน้า', 'อาคาร', 'กลับบ้าน', 'กล่อง', 'กาแฟ', 'ขนม', 'ขนมไทย', 'ขาย', 'ชอบ', 'ซื้อ', 'ดี', 'ดู', 'ทาน', 'น่ารัก', 'น้ำ', 'ปริมาณ', 'มาตรฐาน', 'รสชาติ', 'ราคา', 'สะดวก', 'เครื่องดื่ม', 'เด่น', 'เรียบร้อย', 'เสียบ', 'เหนียว', 'แพง', 'แพ็ค', 'แพ็คเกจ', 'แพ้', 'แวะ', 'ใส่', 'ไม้', 'ของหวาน', 'คน', 'ความปราณี', 'คอ', 'ต', 'ตอน', 'ต่อเติม', 'ทำ', 'ร้านกาแฟ', 'สวย', 'สั่ง', 'ห้อง', 'เชียงใหม่', 'เดี๋ยวนี้', 'เต็ม', 'เมนู', 'แบบนี้', 'แอร์', 'โรตี', '3', 'คนไทย', 'ซอย', 'ตลาด', 'ต่างชาติ', 'ถนน', 'ทั้งวัน', 'นิมมาน', 'ลูกค้า', 'เทพ', 'เมล็ด', 'เลือก', 'แนะนำ', 'กรุงเทพ', 'กิน', 'ขนมเค้ก', 'งาน', 'บรรยากาศ', 'หลากหลาย', 'ออกแบบ', 'อาหาร', 'เก๋', 'เข้มข้น', 'ให้บริการ', 'กลิ่น', 'ช็อต', 'ซุ', 'ท็อป', 'ปั่น', 'ยุ', 'ราด', 'ร้อน', 'ลอง', 'ละเอียด', 'สดชื่น', 'ส้ม', 'หอม', 'หั่น', 'เข้ากัน', 'เที่ยง ๆ', 'เปรี้ยว', 'เย็น', 'เรื่อย ๆ', 'เลมอน', '2', 'ขายของ', 'คุณภาพ', 'จาน', 'ดีมาก', 'ทำได้

In [9]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [10]:
num_topics = 2
chunksize = 15000
passes = 70
iterations = 300
eval_every = 1

temp = dictionary[0]
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, alpha='auto', eta='auto', \
                                    iterations=iterations, num_topics=num_topics, passes=passes, eval_every=eval_every)

CPU times: user 273 ms, sys: 7.33 ms, total: 280 ms
Wall time: 277 ms


In [11]:
# pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

In [12]:
print(f'topic 1 description :{model.show_topic(0)}')
print(f'topic 2 description :{model.show_topic(1)}')

topic 1 description :[('อร่อย', 0.028441254), ('กาแฟ', 0.025114886), ('กิน', 0.025114886), ('อาหาร', 0.01841755), ('จาน', 0.018393531), ('เมนู', 0.015068885), ('หน้า', 0.015068885), ('พิซซ่า', 0.014795531), ('รสชาติ', 0.014707182), ('ดี', 0.014609547)]
topic 2 description :[('ทาน', 0.030944757), ('พิซซ่า', 0.027155576), ('ขนม', 0.022692792), ('แป้ง', 0.022692792), ('กาแฟ', 0.022692792), ('ชอบ', 0.018566806), ('เครื่องดื่ม', 0.018566806), ('ดี', 0.015006786), ('คน', 0.014440822), ('ใส่', 0.010931969)]


In [13]:
reviewFoodComment['topics'] = reviewFoodComment['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
reviewFoodComment['score'] = reviewFoodComment['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [14]:
reviewFoodComment

,Review,Review_tokenized,topics,score
0,คาเฟ่ แห่งหนึ่งย่านบางใหญ่ มีที่จอดรถหน้าร้าน ...,"คาเฟ่,ย่าน,ที่จอดรถ,หน้า,จอด,คัน,ลานจอดรถ,คาเฟ...",0,0.998589
1,ร้านนี้เด่นที่เป็นร้านคาเฟ่กาแฟที่มีขนมไทยขายด...,"เด่น,คาเฟ่,กาแฟ,ขนมไทย,ขาย,แพ็คเกจ,เครื่องดื่ม...",1,0.999105
2,ร้านกาแฟร้านนี้เราทานตั้งแต่ตอนยังไม่ต่อเติมหร...,"ร้านกาแฟ,ทาน,ตอน,ต่อเติม,ห้อง,แอร์,สวย,แบบนี้,...",1,0.998610
3,วันนี้จะมาแนะนำ ร้านกาแฟเทพๆ ย่านถนนนิมมาน ร้า...,"แนะนำ,ร้านกาแฟ,เทพ,ย่าน,ถนน,นิมมาน,หน้า,นิมมาน...",0,0.998737
4,ร้านกาแฟกรุงเทพ บรรยากาศดี มีกาแฟ เครื่องดื่มห...,"ร้านกาแฟ,กรุงเทพ,บรรยากาศ,ดี,กาแฟ,เครื่องดื่ม,...",0,0.999076
5,แนะนำให้มาลองกินกาแฟร้านนี้ เป็นร้านกาแฟเล็ก ๆ...,"แนะนำ,ลอง,กิน,กาแฟ,ร้านกาแฟ,คน,ซื้อ,เรื่อย ๆ,ห...",0,0.999441
6,ร้านอาหารอิตาเลียนที่มีทั้งอาหารเวียดนาม อาหาร...,"ร้านอาหาร,อิตาเลียน,อาหาร,เวียดนาม,อาหารจีน,อา...",0,0.999249
7,พิซซ่าหน้าชีสล้วนๆ อร่อย หวานมัน นัวเนียในปาก ...,"พิซซ่า,หน้า,ชีส,อร่อย,หวาน,นัวเนีย,ปาก,กิน,ตอน...",0,0.999368
8,บอกเลยว่าพิซซ่าเด็ดจริง! ถ้าใครเคยมีโอกาสไปทาน...,"พิซซ่า,เด็ด,!,มีโอกาส,ทาน,อิตาลี,ทาน,พิซซ่า,แท...",1,0.999011
9,ร้านพิซซ่าที่มีเจ้าของเป็นคนเกาหลี ดีกรีแชมป์ห...,"พิซซ่า,มีเจ้าของ,คน,เกาหลี,ดีกรี,แชมป์,หมุน,แป...",1,0.998927


In [15]:
reviewFoodComment.loc[reviewFoodComment["topics"] == 0, 'topics'] = "cafe topic"
reviewFoodComment.loc[reviewFoodComment["topics"] == 1, 'topics'] = "pizza topic"

In [16]:
reviewFoodComment

,Review,Review_tokenized,topics,score
0,คาเฟ่ แห่งหนึ่งย่านบางใหญ่ มีที่จอดรถหน้าร้าน ...,"คาเฟ่,ย่าน,ที่จอดรถ,หน้า,จอด,คัน,ลานจอดรถ,คาเฟ...",cafe topic,0.998589
1,ร้านนี้เด่นที่เป็นร้านคาเฟ่กาแฟที่มีขนมไทยขายด...,"เด่น,คาเฟ่,กาแฟ,ขนมไทย,ขาย,แพ็คเกจ,เครื่องดื่ม...",pizza topic,0.999105
2,ร้านกาแฟร้านนี้เราทานตั้งแต่ตอนยังไม่ต่อเติมหร...,"ร้านกาแฟ,ทาน,ตอน,ต่อเติม,ห้อง,แอร์,สวย,แบบนี้,...",pizza topic,0.998610
3,วันนี้จะมาแนะนำ ร้านกาแฟเทพๆ ย่านถนนนิมมาน ร้า...,"แนะนำ,ร้านกาแฟ,เทพ,ย่าน,ถนน,นิมมาน,หน้า,นิมมาน...",cafe topic,0.998737
4,ร้านกาแฟกรุงเทพ บรรยากาศดี มีกาแฟ เครื่องดื่มห...,"ร้านกาแฟ,กรุงเทพ,บรรยากาศ,ดี,กาแฟ,เครื่องดื่ม,...",cafe topic,0.999076
5,แนะนำให้มาลองกินกาแฟร้านนี้ เป็นร้านกาแฟเล็ก ๆ...,"แนะนำ,ลอง,กิน,กาแฟ,ร้านกาแฟ,คน,ซื้อ,เรื่อย ๆ,ห...",cafe topic,0.999441
6,ร้านอาหารอิตาเลียนที่มีทั้งอาหารเวียดนาม อาหาร...,"ร้านอาหาร,อิตาเลียน,อาหาร,เวียดนาม,อาหารจีน,อา...",cafe topic,0.999249
7,พิซซ่าหน้าชีสล้วนๆ อร่อย หวานมัน นัวเนียในปาก ...,"พิซซ่า,หน้า,ชีส,อร่อย,หวาน,นัวเนีย,ปาก,กิน,ตอน...",cafe topic,0.999368
8,บอกเลยว่าพิซซ่าเด็ดจริง! ถ้าใครเคยมีโอกาสไปทาน...,"พิซซ่า,เด็ด,!,มีโอกาส,ทาน,อิตาลี,ทาน,พิซซ่า,แท...",pizza topic,0.999011
9,ร้านพิซซ่าที่มีเจ้าของเป็นคนเกาหลี ดีกรีแชมป์ห...,"พิซซ่า,มีเจ้าของ,คน,เกาหลี,ดีกรี,แชมป์,หมุน,แป...",pizza topic,0.998927
